# Обучение YOLO для NNIE процессора Hi3559


Исходные материалы:
- https://www.programmersought.com/article/458411298417/
- https://github.com/mahxn0/Yolov5-Hisi3559a-Train

## Подготовка

Действия выполнялись в Ubuntu-22.04 под WSL2. Была установлена [Anaconda](https://docs.anaconda.com/anaconda/install/linux/)

1. Скачать репозиторий YOLOv5
    1. `git clone https://github.com/ultralytics/yolov5.git`
    2. `cd yolov5`
    3. `git reset --hard 69be8e738`

2. Настроить виртуальное окружение в Anaconda
   1. `conda create --name hi_yolov5 python=3.7.9 -y`
   2. `conda activate hi_yolov5`
   3. Отредактировать requirements.txt:
      - удалить строки Coremltools, onnx, Scikit-Learn "#", 
      - добавить строку "onnx-simplifier"
   4. `pip install -r requirements.txt`

## Загрузка датасета

In [13]:
%cd ..


/home/user/projects/otus


In [14]:
%cd yolov5

!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="G17cDggpZioiUyOSqOsJ")
project = rf.workspace("paul-nlfem").project("od_ai")
version = project.version(14)
dataset = version.download("yolov5")

/home/user/projects/otus/yolov5
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to OD_AI-14 in yolov5pytorch:: 100%|██████████| 4598/4598 [00:00<00:00, 8460.68it/s]


In [15]:
YOLO5_PATH = %pwd
print(YOLO5_PATH)
%cat {dataset.location}/data.yaml

/home/user/projects/otus/yolov5
names:
- harvester
- person
- post
- tractor
nc: 4
roboflow:
  license: CC BY 4.0
  project: od_ai
  url: https://universe.roboflow.com/paul-nlfem/od_ai/dataset/14
  version: 14
  workspace: paul-nlfem
test: ../test/images
train: OD_AI-14/train/images
val: OD_AI-14/valid/images


## Модификация архитектуры сети

Опциональные шаги, оставлю для истории, поскольку у нас используется собственный датасет:
> Modify the training parameters and model structure  
>	 A. Modify the number of categories, category names, trains/test/val in the data/coco.yaml file, and modify it according to your own project planning  
>	 B. Modify the number of categories in the Models/YOLOV5s.yaml file  


>	 C. Modify the network structure in the Models/YOLOV5S.yaml, modify the Focus layer to convolutional layer, and set the Stride to 2  
	
  backbone:
```
  # [from, number, module, args]
  # [[-1, 1, Focus, [64, 3]],  # 0-P1/2
  [
   [-1, 1, Conv, [64, 3, 2]],
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, C3, [128]],`
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 9, C3, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, C3, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 1, SPP, [1024, [5, 9, 13]]],
   [-1, 3, C3, [1024, False]],  # 9
  ]
```

     Or use YOLOV5 Release6.0 version has removed the Focus layer,
  	backbone:
```
  # [from, number, module, args]
  [[-1, 1, Conv, [64, 6, 2, 2]],  # 0-P1/2 <--- update
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, C3, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 9, C3, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, C3, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 1, SPP, [1024, [5, 9, 13]]],
   [-1, 3, C3, [1024, False]],  # 9
  ]
```
     Or directly
  https://github.com/mahxn0/Yolov5-Hisi3559a-Train.git

## Обучение кастомной сети на кастомном датасете

In [16]:
%cd $YOLO5_PATH
!python3 train.py --img 416 --batch 16 --epochs 100 --data {dataset.location}/data.yaml --cfg ./models/yolov5s.yaml --weights '' --name yolov5s_results  --cache

/home/user/projects/otus/yolov5
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 616.7/616.7 kB 2.9 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 614.9/614.9 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 614.4/614.4 kB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 606.9/606.9 kB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 606.2/606.2 kB 34.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 605.5/605.5 kB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 605.6/605.6 kB 2.9 MB/s eta 0:00:00a 0:00:01
WARNING ⚠️ Ultralytics settings reset to default values. This may be due to a possible problem with your settings or a recent ultralytics package update. 
View settings with 'yolo settings' or at '/home/user/.config/Ultralytics/settings.yaml'
Update settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'.
requirements: Ultralytics requirement ['gitpython